In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from scipy.stats import pearsonr, spearmanr
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from SentenceComparison.SentenceComparison import SentenceComparison
from ModelHelpers.fill_mask_helpers import get_probability_from_pipeline_for_token
from relation_templates.templates import relations, nominalized_relations, get_length_for_relation, get_templates, get_relation_meta, get_relation_name

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(model(sentence), token)
    return prob

In [6]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]
TOP_K = 1

In [7]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations

In [8]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
my_tokenizer = lambda sentence: tokenizer(sentence)['input_ids']

In [10]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [11]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 34844.90it/s]


In [12]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=500)

In [13]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(unmasker(sentence), token)
    return prob

In [14]:
Comparer = SentenceComparison(relations, get_templates, metric, MASK, get_relation_meta)
Comparer.compare(TREx.data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 5748.70it/s]


In [15]:
accuracies = querier.results_for_calculation()
np_accuracies = np.array(accuracies)[1:,1:].flatten().astype(float)

In [16]:
probabilities = Comparer.make_global_comparison()
np_probabilities = np.array(probabilities)[:,1:].flatten().astype(float)

In [17]:
lengths = get_length_for_relation(my_tokenizer)
np_lengths = np.array(lengths)[1:,1:].flatten().astype(float)

Accuracy vs Length

In [18]:
r_corr , _ = pearsonr(np_accuracies, np_lengths)
r_corr

-0.11394431561611935

In [19]:
r_corr , _ = spearmanr(np_accuracies, np_lengths)
r_corr

-0.17139819354264338

Probability vs Length

In [20]:
r_corr , _ = pearsonr(np_probabilities, np_lengths)
r_corr

-0.13602374741449863

In [21]:
r_corr , _ = spearmanr(np_probabilities, np_lengths)
r_corr

-0.18548964925361447

In [22]:
for row in lengths:
    print(" & ".join(map(str, row))+" \\\\")

relations & simple & compound & complex & compound-complex \\
P1001 & 11 & 13 & 15 & 20 \\
P101 & 11 & 18 & 19 & 23 \\
P103 & 11 & 14 & 13 & 18 \\
P106 & 10 & 18 & 12 & 27 \\
P108 & 8 & 14 & 12 & 18 \\
P127 & 9 & 13 & 11 & 19 \\
P1303 & 7 & 15 & 17 & 21 \\
P131 & 9 & 14 & 13 & 18 \\
P136 & 8 & 13 & 15 & 23 \\
P1376 & 10 & 16 & 14 & 19 \\
P138 & 9 & 17 & 16 & 22 \\
P140 & 12 & 18 & 17 & 28 \\
P1412 & 10 & 15 & 15 & 21 \\
P159 & 13 & 16 & 15 & 20 \\
P17 & 9 & 13 & 14 & 19 \\
P176 & 9 & 15 & 12 & 19 \\
P178 & 9 & 17 & 11 & 22 \\
P19 & 9 & 16 & 16 & 23 \\
P190 & 10 & 16 & 19 & 23 \\
P20 & 8 & 15 & 15 & 20 \\
P264 & 11 & 13 & 13 & 21 \\
P27 & 8 & 15 & 14 & 20 \\
P276 & 9 & 14 & 13 & 18 \\
P279 & 12 & 17 & 16 & 18 \\
P30 & 9 & 13 & 13 & 18 \\
P31 & 8 & 16 & 14 & 19 \\
P36 & 10 & 15 & 15 & 19 \\
P361 & 9 & 15 & 13 & 18 \\
P364 & 11 & 16 & 15 & 22 \\
P37 & 11 & 16 & 15 & 20 \\
P39 & 10 & 14 & 13 & 18 \\
P407 & 9 & 15 & 15 & 22 \\
P413 & 9 & 13 & 12 & 20 \\
P449 & 10 & 13 & 13 & 21 \\
P463 & 10

### simple vs ...

In [23]:
keys = ["Compound", "Complex", "Compound-Complex"]
key_accuracies = {key:np.array(accuracies)[1:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}
key_probabilities = {key:np.array(probabilities)[:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}
key_lengths = {key:np.array(lengths)[1:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}

In [24]:
for c_key in keys:
    r_ac, _ = pearsonr(key_accuracies[c_key], key_lengths[c_key])
    r_prob, _ = pearsonr(key_probabilities[c_key], key_lengths[c_key])
    print(f"Simple vs {c_key} - accuracy: {r_ac} - probability: {r_prob}")

Simple vs Compound - accuracy: -0.15397431234672393 - probability: -0.20627475551013336
Simple vs Complex - accuracy: -0.14701360627813553 - probability: -0.23762542524874958
Simple vs Compound-Complex - accuracy: -0.2162803919536829 - probability: -0.20431758095959707


In [25]:
for c_key in keys:
    r_ac, _ = spearmanr(key_accuracies[c_key], key_lengths[c_key])
    r_prob, _ = spearmanr(key_probabilities[c_key], key_lengths[c_key])
    print(f"Simple vs {c_key} - accuracy: {r_ac} - probability: {r_prob}")

Simple vs Compound - accuracy: -0.2250443432701609 - probability: -0.2974020815327769
Simple vs Complex - accuracy: -0.20129127602493 - probability: -0.37854853118233434
Simple vs Compound-Complex - accuracy: -0.23671638764971448 - probability: -0.24311980758242166


In [26]:
for c_key in keys:
    r_ac, _ = pearsonr(key_accuracies[c_key], key_lengths[c_key])
    s_ac, _ = spearmanr(key_accuracies[c_key], key_lengths[c_key])
    print(f"10 & Simple and {c_key} & {r_ac:.4f} & {s_ac:.4f} \\\\")

10 & Simple and Compound & -0.1540 & -0.2250 \\
10 & Simple and Complex & -0.1470 & -0.2013 \\
10 & Simple and Compound-Complex & -0.2163 & -0.2367 \\
